К ответу на вопрос https://ru.stackoverflow.com/questions/1268254 Вычисление порождающей точки в группе точек эллиптической кривой


In [30]:
from ecpy.curves     import Curve,Point

In [46]:
cv = Curve.get_curve('secp256k1')
P  = Point(0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798,
       0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8,
           cv)
k  = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141
Q = 5*P
inv_5 = pow(5, k-2,k)
R  = inv_5*Q

In [47]:
R == P

True

In [48]:
R==Q

False

In [49]:
hex(inv_5)

'0x66666666666666666666666666666665e445f1f5dfb6a67e4cba8c385348e6e7'

Проверка тестового примера топик стартера - умножение на 5 точки c x-координатой `55066263022277343669578718895168534326250603453777594175500187360389116729240`

(на самом деле это тот же самый пример с точкой Bitcoin, но в десятичном виде - я просто не узнал эту точку, поэтому вычислял y по-честному)

In [84]:
# Вычисление квадратного корня по модулю p
# https://eli.thegreenplace.net/2009/03/07/computing-modular-square-roots-in-python
def modular_sqrt(a, p):

    def legendre_symbol(a, p):
        """ Compute the Legendre symbol a|p using
            Euler's criterion. p is a prime, a is
            relatively prime to p (if p divides
            a, then a|p = 0)
            Returns 1 if a has a square root modulo
            p, -1 otherwise.
        """
        ls = pow(a, (p - 1) // 2, p)
        return -1 if ls == p - 1 else ls

    """ Find a quadratic residue (mod p) of 'a'. p
        must be an odd prime.
        Solve the congruence of the form:
            x^2 = a (mod p)
        And returns x. Note that p - x is also a root.
        0 is returned is no square root exists for
        these a and p.
        The Tonelli-Shanks algorithm is used (except
        for some simple cases in which the solution
        is known from an identity). This algorithm
        runs in polynomial time (unless the
        generalized Riemann hypothesis is false).
    """
    # Simple cases
    #
    if legendre_symbol(a, p) != 1:
        return 0
    elif a == 0:
        return 0
    elif p == 2:
        return p
    elif p % 4 == 3:
        return pow(a, (p + 1) // 4, p)

    # Partition p-1 to s * 2^e for an odd s (i.e.
    # reduce all the powers of 2 from p-1)
    #
    s = p - 1
    e = 0
    while s % 2 == 0:
        s //= 2
        e += 1

    # Find some 'n' with a legendre symbol n|p = -1.
    # Shouldn't take long.
    #
    n = 2
    while legendre_symbol(n, p) != -1:
        n += 1

    # Here be dragons!
    # Read the paper "Square roots from 1; 24, 51,
    # 10 to Dan Shanks" by Ezra Brown for more
    # information
    #

    # x is a guess of the square root that gets better
    # with each iteration.
    # b is the "fudge factor" - by how much we're off
    # with the guess. The invariant x^2 = ab (mod p)
    # is maintained throughout the loop.
    # g is used for successive powers of n to update
    # both a and b
    # r is the exponent - decreases with each update
    #
    x = pow(a, (s + 1) // 2, p)
    b = pow(a, s, p)
    g = pow(n, s, p)
    r = e

    while True:
        t = b
        m = 0
        for m in range(r):
            if t == 1:
                break
            t = pow(t, 2, p)

        if m == 0:
            return x

        gs = pow(g, 2 ** (r - m - 1), p)
        g = (gs * gs) % p
        x = (x * gs) % p
        b = (b * g) % p
        r = m

In [85]:
# проверка 4^2 mod 11 = 5
modular_sqrt(5, 11)

4

In [86]:
p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
def secp256k1Fn(x):
    "Вычисляет y координату по х координате для точки на secp256k1"
    y_square = (x*x*x+7)%p
    return modular_sqrt(y_square, p)

Проверка для известной точки

In [87]:
x0 = 0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798
y0 = 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8
y0 - secp256k1Fn(x0)

0

In [88]:
(x0*x0*x0+7-y0*y0)%p

0

Тестовая точка из примера автора вопроса

In [89]:
x1 = 55066263022277343669578718895168534326250603453777594175500187360389116729240
y1 = secp256k1Fn(x1)
y1

32670510020758816978083085130507043184471273380659243275938904335757337482424

In [90]:
test_P = Point(x1,y1, curve=cv, check=True)
test_Q = 5*test_P
test_Q.x

21505829891763648114329055987619236494102133314575206970830385799158076338148

In [91]:
test_R = test_Q*inv_5

In [76]:
test_R == test_P

True